In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from splinter import Browser

In [2]:
#we visit the first site and get our title/paragraph text
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
driver = webdriver.Chrome()
driver.get(url)
data = driver.page_source
driver.quit()
soup = bs(data, 'html.parser')

In [3]:
news_title = soup.find('div','content_title').text
news_p = soup.find('div','article_teaser_body').text
print(news_title)
print(news_p)

Media Get a Close-Up of NASA's Mars 2020 Rover
The clean room at NASA's Jet Propulsion Laboratory was open to the media to see NASA's next Mars explorer before it leaves for Florida in preparation for a summertime launch.


In [4]:
#we visit the second site and navigate to our image page and save the url
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser = Browser('chrome', 'chromedriver.exe', headless=False)
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
browser.click_link_by_partial_text('FULL IMAGE')
browser.click_link_by_partial_text('more info')
browser.click_link_by_partial_href('/spaceimages/images/largesize')
featured_image_url = browser.url
print(featured_image_url)
browser.quit()

C:\Users\maste\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,
C:\Users\maste\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:520: FutureWarning: browser.find_link_by_partial_href is deprecated. Use browser.links.find_by_partial_href instead.
  FutureWarning,


https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19920_hires.jpg


In [5]:
#we visit our third site and retrieve the text of the latest tweet
url = "https://twitter.com/marswxreport?lang=en"
response = requests.get(url)
soup = bs(response.text, 'html.parser')
tweet_text = soup.find('p','tweet-text').text
#don't forget to remove the image text! If it exists, this will remove it.
img_text = soup.find('a','u-hidden').text
mars_weather = tweet_text.replace(img_text,'')
print(mars_weather)

InSight sol 400 (2020-01-11) low -99.1ºC (-146.5ºF) high -15.7ºC (3.8ºF)
winds from the SSE at 5.5 m/s (12.3 mph) gusting to 22.3 m/s (49.9 mph)
pressure at 6.40 hPa


In [6]:
#we visit our fourth site and retrieve the needed table
url = "https://space-facts.com/mars/"
tables = pd.read_html(url)
df = tables[0].rename(columns={0:'Metric', 1:'Value'})
df = df.set_index('Metric')
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Metric</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </

In [7]:
#we visit our final site to retrieve our image urls
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser = Browser('chrome', 'chromedriver.exe', headless=False)
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
divs = soup.find_all('div', 'description')

In [8]:
hemisphere_image_urls = []
for div in divs:
    link = div.find('a')
    browser.visit("https://astrogeology.usgs.gov/"+link['href'])
    title = link.text.replace(" Enhanced","")
    html = browser.html
    soup = bs(html, 'html.parser')
    img_url = soup.find('a', text="Sample")['href']
    hemisphere_image_urls.append({"title":title, "img_url":img_url})
browser.quit()
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
